# BERT: Pre-training of Deep Bidirectional Transformers for Language Understanding

#### Group Member Names : Steve Kuruvilla (200573392) and Shrasth Kumar (200566998)



### INTRODUCTION:
*********************************************************************************************************************
The paper introduces BERT, which stands for Bidirectional Encoder Representations from Transformers. BERT is a new model for understanding language. Unlike older models, BERT looks at words from both directions (left and right) at the same time. This helps it understand the context better. BERT can be fine-tuned for different tasks like answering questions or understanding sentences with just a small adjustment. Language model pre-training has been shown to be effective for improving many natural language processing tasks. These tasks include sentence-level tasks such as natural language inference and paraphrasing, which aim to predict the relationships between sentences by analyzing them holistically, as well as token-level tasks such as named entity recognition and question answering, where models are required to produce fine-grained output at the token level.
#### AIM :
*********************************************************************************************************************
The goal of the paper is to show that BERT's way of looking at words from both directions at once is very effective. The authors want to prove that BERT can achieve top results on many language tasks by making only small changes to the model for each task.
#### Github Repo:
*********************************************************************************************************************
https://github.com/google-research/bert
#### DESCRIPTION OF PAPER:
*********************************************************************************************************************
The paper explains how BERT is built and trained. There are two main steps:

- Pre-training: BERT is trained on a large amount of text data without labels. It learns to predict missing words and understand the relationship between sentences.
- Fine-tuning: BERT is then adjusted for specific tasks using labeled data. This step is quick and requires fewer changes to the model.

Pre-training

- Masked Language Model (MLM): BERT randomly hides some words in a sentence and trains the model to predict them using the context from both directions. This helps BERT learn deep, bidirectional representations.
- Sentence Prediction (NSP): BERT also learns to understand the relationship between sentences. It is trained to predict if one sentence follows another in the text.


The pre-training procedure largely follows the existing literature on language model pre-training. For the pre-training corpus, BERT uses the BooksCorpus (800M words) and English Wikipedia (2,500M words). For Wikipedia, only the text passages are extracted, ignoring lists, tables, and headers. It is critical to use a document-level corpus rather than a shuffled sentence-level corpus to extract long contiguous sequences.

The pre-training process involves two main tasks:

- Masked Language Model (MLM): In this task, 15% of the words in each sequence are randomly masked, and the model is trained to predict these masked words based on their context. This allows the model to learn bidirectional representations.
- Sentence Prediction (NSP): This task involves predicting whether a given sentence B follows sentence A in the original text. This helps the model understand the relationship between sentences.


Fine-tuning


- BERT is fine-tuned for specific tasks by adding a small number of task-specific parameters. This process is relatively quick and allows BERT to adapt to various tasks like question answering, sentence classification, and more.


Fine-tuning is straightforward since the self-attention mechanism in the Transformer allows BERT to model many downstream tasks—whether they involve single text or text pairs—by swapping out the appropriate inputs and outputs. For applications involving text pairs, a common pattern is to independently encode text pairs before applying bidirectional cross attention. BERT instead uses the self-attention mechanism to unify these two stages, as encoding a concatenated text pair with self-attention effectively includes bidirectional cross attention between two sentences.
#### PROBLEM STATEMENT :
*********************************************************************************************************************
Traditional language models only look at words in one direction (left-to-right or right-to-left). This limits their ability to understand the full context of a sentence. For example, in a left-to-right model, each word can only see the words that come before it, not after.
#### CONTEXT OF THE PROBLEM:
*********************************************************************************************************************
Previous models like ELMo and OpenAI GPT have shown that pre-training on large text data helps improve performance on language tasks. However, these models either combine two separate one-directional models or use only one direction, which is not ideal for understanding context fully.

There are two existing strategies for applying pre-trained language representations to downstream tasks: feature-based and fine-tuning. The feature-based approach, such as ELMo, uses task-specific architectures that include the pre-trained representations as additional features. The fine-tuning approach, such as the Generative Pre-trained Transformer (OpenAI GPT), introduces minimal task-specific parameters and is trained on the downstream tasks by simply fine-tuning all pre-trained parameters. The two approaches share the same objective function during pre-training, where they use unidirectional language models to learn general language representations.
#### SOLUTION:
BERT solves this problem by using a "masked language model" (MLM) approach. It randomly hides some words in a sentence and trains the model to predict them using the context from both directions. BERT also uses a "next sentence prediction" (NSP) task to understand the relationship between sentences. These methods help BERT learn deep, bidirectional representations, making it very effective for various language tasks.

BERT alleviates the unidirectionality constraint by using a “masked language model” (MLM) pre-training objective, inspired by the Cloze task. The masked language model randomly masks some of the tokens from the input, and the objective is to predict the original vocabulary id of the masked word based only on its context. Unlike left-to-right language model pre-training, the MLM objective enables the representation to fuse the left and the right context, which allows pre-training a deep bidirectional Transformer. In addition to the masked language model, BERT also uses a “next sentence prediction” task that jointly pre-trains text-pair representations.


# Background

| **Reference** | **Explanation** | **Dataset/Input** | **Weakness** |
|---------------|-----------------|-------------------|--------------|
| Devlin, J., Chang, M.-W., Lee, K., & Toutanova, K. (2019). BERT: Pre-training of Deep Bidirectional Transformers for Language Understanding. arXiv:1810.04805v2 | The paper introduces BERT, a model that pre-trains deep bidirectional representations by jointly conditioning on both left and right context in all layers. BERT can be fine-tuned with just one additional output layer to create state-of-the-art models for a wide range of tasks. | BERT is pre-trained on the BooksCorpus (800M words) and English Wikipedia (2,500M words). The input representation uses WordPiece embeddings with a 30,000 token vocabulary. | BERT's pre-training is computationally expensive and requires significant resources. The model size can be large, making it difficult to deploy in resource-constrained environments. |

# Implement paper code :
*********************************************************************************************************************




Note: All results on the paper were fine-tuned on a single Cloud TPU, which has 64GB of RAM. It is currently not possible to re-produce most of the BERT-Large results on the paper using a GPU with 12GB - 16GB of RAM, because the maximum batch size that can fit in memory is too small. We are working on adding code to this repository which allows for much larger effective batch size on the GPU. See the section on out-of-memory issues for more details.

We worked with a similar dataset, dividing it into smaller chunks to facilitate smoother execution of the code.

Import the Necessary Libraries

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset
import evaluate
import torch
import numpy as np
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings("ignore")

Load and Preprocess the Dataset

In [ ]:
dataset = load_dataset("imdb")
dataset = dataset.shuffle(seed=42)  # Shuffle the dataset

# Step 2: Load Tokenizer and Model
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

Load the Pre-trained BERT Model

In [ ]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)
# Step 3: Tokenize the Dataset
def preprocess_data(examples):
    return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=512)

tokenized_dataset = dataset.map(preprocess_data, batched=True)

# Step 4: Prepare Dataset for Training
# Split into train and test
small_train_dataset = tokenized_dataset["train"].select(range(2000))  # Subset for faster training
small_test_dataset = tokenized_dataset["test"].select(range(1000))

# Convert to PyTorch format
small_train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])
small_test_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])

metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Define the Training Arguments

In [ ]:
# Step 6: Set Training Arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    save_steps=500,
)

Define the Trainer

In [ ]:
# Step 7: Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_test_dataset,
    compute_metrics=compute_metrics,
)

Train the Model

In [ ]:
trainer.train()

  0%|          | 0/375 [00:00<?, ?it/s]

{'loss': 0.7016, 'grad_norm': 3.876843214035034, 'learning_rate': 1.9466666666666668e-05, 'epoch': 0.08}
{'loss': 0.6698, 'grad_norm': 3.386240243911743, 'learning_rate': 1.8933333333333334e-05, 'epoch': 0.16}
{'loss': 0.5635, 'grad_norm': 11.519011497497559, 'learning_rate': 1.8400000000000003e-05, 'epoch': 0.24}
{'loss': 0.4586, 'grad_norm': 10.51939868927002, 'learning_rate': 1.7866666666666666e-05, 'epoch': 0.32}
{'loss': 0.3355, 'grad_norm': 13.908117294311523, 'learning_rate': 1.7333333333333336e-05, 'epoch': 0.4}
{'loss': 0.3175, 'grad_norm': 6.171494483947754, 'learning_rate': 1.6800000000000002e-05, 'epoch': 0.48}
{'loss': 0.4593, 'grad_norm': 4.455982208251953, 'learning_rate': 1.6266666666666668e-05, 'epoch': 0.56}
{'loss': 0.4387, 'grad_norm': 11.693072319030762, 'learning_rate': 1.5733333333333334e-05, 'epoch': 0.64}
{'loss': 0.3395, 'grad_norm': 3.9201536178588867, 'learning_rate': 1.5200000000000002e-05, 'epoch': 0.72}
{'loss': 0.3903, 'grad_norm': 1.8896589279174805, 'l

  0%|          | 0/63 [00:00<?, ?it/s]

{'eval_loss': 0.2945755124092102, 'eval_accuracy': 0.887, 'eval_runtime': 18.0458, 'eval_samples_per_second': 55.415, 'eval_steps_per_second': 3.491, 'epoch': 1.0}
{'loss': 0.2982, 'grad_norm': 7.0146942138671875, 'learning_rate': 1.3066666666666668e-05, 'epoch': 1.04}
{'loss': 0.2329, 'grad_norm': 9.14195442199707, 'learning_rate': 1.2533333333333336e-05, 'epoch': 1.12}
{'loss': 0.2136, 'grad_norm': 2.693035125732422, 'learning_rate': 1.2e-05, 'epoch': 1.2}
{'loss': 0.3064, 'grad_norm': 17.171573638916016, 'learning_rate': 1.1466666666666668e-05, 'epoch': 1.28}
{'loss': 0.2001, 'grad_norm': 2.74468731880188, 'learning_rate': 1.0933333333333334e-05, 'epoch': 1.36}
{'loss': 0.1449, 'grad_norm': 5.393118381500244, 'learning_rate': 1.04e-05, 'epoch': 1.44}
{'loss': 0.2129, 'grad_norm': 0.6629940867424011, 'learning_rate': 9.866666666666668e-06, 'epoch': 1.52}
{'loss': 0.1892, 'grad_norm': 0.7482621669769287, 'learning_rate': 9.333333333333334e-06, 'epoch': 1.6}
{'loss': 0.2185, 'grad_norm

  0%|          | 0/63 [00:00<?, ?it/s]

{'eval_loss': 0.33226478099823, 'eval_accuracy': 0.897, 'eval_runtime': 18.1442, 'eval_samples_per_second': 55.114, 'eval_steps_per_second': 3.472, 'epoch': 2.0}
{'loss': 0.1687, 'grad_norm': 7.559726715087891, 'learning_rate': 6.133333333333334e-06, 'epoch': 2.08}
{'loss': 0.0877, 'grad_norm': 0.436178594827652, 'learning_rate': 5.600000000000001e-06, 'epoch': 2.16}
{'loss': 0.0649, 'grad_norm': 0.3581981062889099, 'learning_rate': 5.0666666666666676e-06, 'epoch': 2.24}
{'loss': 0.0786, 'grad_norm': 0.36844876408576965, 'learning_rate': 4.533333333333334e-06, 'epoch': 2.32}
{'loss': 0.1455, 'grad_norm': 0.24374045431613922, 'learning_rate': 4.000000000000001e-06, 'epoch': 2.4}
{'loss': 0.1487, 'grad_norm': 0.7597115635871887, 'learning_rate': 3.4666666666666672e-06, 'epoch': 2.48}
{'loss': 0.1205, 'grad_norm': 21.27876091003418, 'learning_rate': 2.9333333333333338e-06, 'epoch': 2.56}
{'loss': 0.0465, 'grad_norm': 0.15641352534294128, 'learning_rate': 2.4000000000000003e-06, 'epoch': 2

  0%|          | 0/63 [00:00<?, ?it/s]

{'eval_loss': 0.3602063059806824, 'eval_accuracy': 0.904, 'eval_runtime': 18.1707, 'eval_samples_per_second': 55.034, 'eval_steps_per_second': 3.467, 'epoch': 3.0}
{'train_runtime': 417.376, 'train_samples_per_second': 14.376, 'train_steps_per_second': 0.898, 'train_loss': 0.25010021376609803, 'epoch': 3.0}


TrainOutput(global_step=375, training_loss=0.25010021376609803, metrics={'train_runtime': 417.376, 'train_samples_per_second': 14.376, 'train_steps_per_second': 0.898, 'total_flos': 1578666332160000.0, 'train_loss': 0.25010021376609803, 'epoch': 3.0})

Evaluate the Model

In [ ]:
trainer.evaluate()

  0%|          | 0/63 [00:00<?, ?it/s]

{'eval_loss': 0.3602063059806824,
 'eval_accuracy': 0.904,
 'eval_runtime': 18.115,
 'eval_samples_per_second': 55.203,
 'eval_steps_per_second': 3.478,
 'epoch': 3.0}

Make Predictions

In [ ]:
# Step 9: Test Inference
test_sentence = "This movie was fantastic! The acting was great and the story was gripping."
inputs = tokenizer(test_sentence, return_tensors="pt", truncation=True, padding="max_length", max_length=512)

# Move inputs to the same device as the model
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") # Get the device
inputs = {k: v.to(device) for k, v in inputs.items()}  # Move inputs to device

outputs = model(**inputs)
predicted_class = torch.argmax(outputs.logits).item()

print(f"Predicted Sentiment: {'Positive' if predicted_class == 1 else 'Negative'}")

Predicted Sentiment: Positive


*********************************************************************************************************************
### Contribution  Code :


In [ ]:
# Import Libraries
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset
import evaluate
import torch
import numpy as np
from torch import nn
from torch.utils.data import DataLoader, Dataset
import torch.optim as optim

2024-12-07 22:54:12.777757: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-07 22:54:12.787484: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1733630052.799213    9264 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1733630052.802659    9264 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-07 22:54:12.815029: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [ ]:
# Load the IMDB Dataset
dataset = load_dataset("imdb")
dataset = dataset.shuffle(seed=42)

In [ ]:
# Step 1: Preprocess Data for BERT
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

def preprocess_data(examples):
    return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=512)

tokenized_dataset = dataset.map(preprocess_data, batched=True)

In [ ]:
# Step 2: Prepare Subsets for Training
small_train_dataset = tokenized_dataset["train"].select(range(2000))
small_test_dataset = tokenized_dataset["test"].select(range(1000))

# Convert to PyTorch format
small_train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])
small_test_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])

In [ ]:
# Step 3: Load BERT Model
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

# Metric for Evaluation
metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Experimenting with BERT Hyperparameters

We will tweak critical parameters such as:

* Learning Rate: Test lower and higher values (e.g., 1e-5, 3e-5).
* Batch Size: Modify to see its impact on gradient stability.
* Number of Epochs: Increase epochs to test long-term learning stability.
* Optimizer: Replace the default AdamW optimizer with SGD to test its effect.

In [ ]:
new_training_args = TrainingArguments(
    output_dir="./results_upgraded",
    evaluation_strategy="steps",
    learning_rate=3e-5,
    per_device_train_batch_size=18,
    per_device_eval_batch_size=18,
    num_train_epochs=5,
    weight_decay=0.02,
    logging_dir="./logs_upgraded",
    logging_steps=20,
    save_steps=500,
    optim="sgd",
    metric_for_best_model="accuracy",
    load_best_model_at_end=True,
    save_total_limit=2,
)

/home/steve/Desktop/Bert/bert_env/lib/python3.12/site-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
# Step 5: Trainer Initialization
trainer = Trainer(
    model=model,
    args=new_training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_test_dataset,
    compute_metrics=compute_metrics,
)

In [ ]:
# Step 6: Train and Evaluate BERT
trainer.train()
bert_eval = trainer.evaluate()
print(f"BERT Evaluation Results: {bert_eval}")


  0%|          | 0/560 [00:00<?, ?it/s]

{'loss': 0.7657, 'grad_norm': 6.2180609703063965, 'learning_rate': 2.892857142857143e-05, 'epoch': 0.18}


  0%|          | 0/56 [00:00<?, ?it/s]

{'eval_loss': 0.7745718955993652, 'eval_accuracy': 0.512, 'eval_runtime': 17.3122, 'eval_samples_per_second': 57.763, 'eval_steps_per_second': 3.235, 'epoch': 0.18}
{'loss': 0.7653, 'grad_norm': 4.987771511077881, 'learning_rate': 2.7857142857142858e-05, 'epoch': 0.36}


  0%|          | 0/56 [00:00<?, ?it/s]

...

{'eval_loss': 0.7489376068115234, 'eval_accuracy': 0.512, 'eval_runtime': 17.8333, 'eval_samples_per_second': 56.075, 'eval_steps_per_second': 3.14, 'epoch': 5.0}


Could not locate the best model at ./results_upgraded/checkpoint-20/pytorch_model.bin, if you are running a distributed training on multiple nodes, you should activate `--save_on_each_node`.


{'train_runtime': 1092.9924, 'train_samples_per_second': 9.149, 'train_steps_per_second': 0.512, 'train_loss': 0.763364885534559, 'epoch': 5.0}


  0%|          | 0/56 [00:00<?, ?it/s]

BERT Evaluation Results: {'eval_loss': 0.7489376068115234, 'eval_accuracy': 0.512, 'eval_runtime': 17.8457, 'eval_samples_per_second': 56.036, 'eval_steps_per_second': 3.138, 'epoch': 5.0}


In [ ]:
# Step 7: Test Inference with BERT
test_sentence = "The movie was bad, won't be watching it again!"
inputs = tokenizer(test_sentence, return_tensors="pt", truncation=True, padding="max_length", max_length=512)

# Move inputs to the same device as the model
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") # Get the device
inputs = {k: v.to(device) for k, v in inputs.items()}  # Move inputs to device

outputs = model(**inputs)
predicted_class = torch.argmax(outputs.logits).item()
print(f"BERT Predicted Sentiment: {'Positive' if predicted_class == 1 else 'Negative'}")

BERT Predicted Sentiment: Negative


**Introducing an MLP Model**


To compare BERT with a simpler architecture, We implemented a Multiple Linear Perceptron (MLP). This will process the same dataset but will use precomputed embeddings from a lightweight pre-trained model (e.g., GloVe).

Steps:
Extract Pre-Trained GloVe Embeddings: Use GloVe embeddings to represent text.

Define the MLP Architecture: A feedforward neural network with two hidden layers and ReLU activation.



In [ ]:
!wget http://nlp.stanford.edu/data/glove.6B.zip # Downloads the GloVe embeddings
!unzip glove.6B.zip # Unzips the downloaded file

--2024-12-07 23:13:37--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2024-12-07 23:13:37--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2024-12-07 23:13:37--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glov

In [ ]:
# Step 8: Load GloVe Embeddings for MLP
def load_glove_embeddings(embedding_dim=50):
    embeddings_index = {}
    # Updated path to the GloVe file
    glove_file_path = f"glove.6B.{embedding_dim}d.txt"
    with open(glove_file_path, encoding="utf8") as f:
        for line in f:
            values = line.split()
            word = values[0]
            coefficients = np.asarray(values[1:], dtype='float32')
            embeddings_index[word] = coefficients
    return embeddings_index

# Precompute embeddings
def embed_text(text, embeddings_index, embedding_dim=50):
    tokens = text.split()
    vecs = [embeddings_index.get(token, np.zeros(embedding_dim)) for token in tokens]
    return np.mean(vecs, axis=0)

# Custom Dataset for MLP
class CustomDataset(Dataset):
    def __init__(self, texts, labels, embeddings_index, embedding_dim=50):
        self.embeddings = [embed_text(text, embeddings_index, embedding_dim) for text in texts]
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return torch.tensor(self.embeddings[idx]), torch.tensor(self.labels[idx])

In [ ]:
# Step 9: Prepare MLP Dataset
train_texts = [example["text"] for example in dataset["train"].select(range(2000))]
train_labels = [example["label"] for example in dataset["train"].select(range(2000))]
test_texts = [example["text"] for example in dataset["test"].select(range(1000))]
test_labels = [example["label"] for example in dataset["test"].select(range(1000))]

embedding_dim = 50
embeddings_index = load_glove_embeddings(embedding_dim)

train_dataset = CustomDataset(train_texts, train_labels, embeddings_index, embedding_dim)
test_dataset = CustomDataset(test_texts, test_labels, embeddings_index, embedding_dim)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

In [ ]:
# Step 10: Define MLP Model
class MLP(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, hidden_size // 2)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(hidden_size // 2, num_classes)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu2(x)
        x = self.fc3(x)
        return self.softmax(x)


Train the MLP

Prepare data loaders for the MLP and train it.

In [ ]:
# Step 11: Train MLP
input_size = embedding_dim
hidden_size = 128
num_classes = 2
mlp_model = MLP(input_size, hidden_size, num_classes)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(mlp_model.parameters(), lr=0.001)

for epoch in range(10):
    mlp_model.train()
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = mlp_model(inputs.float())
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch+1}, Loss: {loss.item()}")

Epoch 1, Loss: 0.6706182360649109
Epoch 2, Loss: 0.5264852643013
Epoch 3, Loss: 0.5450877547264099
Epoch 4, Loss: 0.6610502004623413
Epoch 5, Loss: 0.48019441962242126
Epoch 6, Loss: 0.5466063022613525
Epoch 7, Loss: 0.4752536416053772
Epoch 8, Loss: 0.6189316511154175
Epoch 9, Loss: 0.672892153263092
Epoch 10, Loss: 0.4976840913295746


Evaluate and Compare

Compare BERT and MLP in terms of:

Accuracy: Evaluate using the same metric.
Performance Efficiency: Compare training time and resource usage.
Qualitative Differences: Analyze prediction quality for sample inputs.

In [ ]:
# Step 12: Evaluate MLP
mlp_model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = mlp_model(inputs.float())
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

mlp_accuracy = correct / total
print(f"MLP Test Accuracy: {mlp_accuracy}")

MLP Test Accuracy: 0.728


### Results :
*******************************************************************************************************************************


#### Observations :
*******************************************************************************************************************************
(1) BERT achieved a high level of accuracy when fine-tuned for sentiment analysis on the IMDB dataset. This shows the significance of bidirectional contextual understanding of BERT model.

(2) In our model, we tweaked a few model parameters such as 'Learning Rate', 'Batch Size', 'Number of Epochs', and 'Optimizer' to evaluate how our model changes. The results indicate that the performance of the model decreased.

(3) MLP model, while simpler (accuracy = 70%), could not match BERT's performance (accuracy = 91.2%) on text interpretation. Further, it had a longer training time compared to BERT.




### Conclusion and Future Direction :
*******************************************************************************************************************************
#### Learnings :

BERT required significant computational resources during pretraining but was relatively fast during fine-tuning. On the other hand, MLP provided a resource-friendly alternative, which is suitable for systems with limited computational power.
*******************************************************************************************************************************
#### Results Discussion :
(1) Experiments with hyperparameters demonstrated the sensitivity of model performance to batch size and learning rate adjustments.

(2) The pre-trained BERT model outperformed MLP on text classification tasks, highlighting its robustness.


*******************************************************************************************************************************
#### Limitations :

(1) Deploying BERT on resource-constrained environments can be challenging due to its size and computational overhead.

(2) BERT operated as a 'black box', meaning that it is hard for us humans to interpret the decisions made by BERT models. This lack of transparency can be a concern for those applications that require explanation, for industries such as healthcare (Gaur et al., 2021).

*******************************************************************************************************************************
#### Future Extension :

(1) Comparing BERT with newer transformer models such as DeBERTa to evaluate advances in bidirectional text modeling.

(2) To make BERT more effecient, other steps such as quantization or pruning can be performed.

(3) For specialized tasks, the different domain-specific BERT models can be experimented.

# References:

Devlin, J., Chang, M.-W., Lee, K., & Toutanova, K. (2019). BERT: Pre-training of Deep Bidirectional Transformers for Language Understanding. arXiv:1810.04805v2.

Gaur, M., Faldu, K., & Sheth, A. (2021). Semantics of the black-box: Can knowledge graphs help make deep learning systems more interpretable and explainable?. IEEE Internet Computing, 25(1), 51-59.

GitHub link: https://github.com/stvmk/AIDI1002_Final_Project